In [1]:
import cv2
import numpy as np
import scipy.misc
import os,numpy as np
import time
os.chdir("/home/mckc/Imagedb/")
import uuid


face_cascade = cv2.CascadeClassifier('/home/mckc/Downloads/opencv-2.4.13/data/haarcascades_GPU/haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)

In [ ]:
import cPickle
def standard(X):
    return (X - X.mean())/X.max()

def Pre_Process(face):
    from skimage.transform import resize
    X = standard(resize(face,(96,96))).reshape(-1,1,96,96)
    X_normal = X.reshape(-1,9216)
    return X,X_normal

# load it again
with open('/home/mckc/nolearn_model.pkl', 'rb') as fid:
    Net = cPickle.load(fid)
    
map = np.load('/home/mckc/map.npy')

Using gpu device 0: Quadro M2000M (CNMeM is enabled with initial size: 10 MB, cuDNN 5005)


In [ ]:
while True:
    # Capture frame-by-frame
    #time.sleep(1)
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #gray = cv2.equalizeHist(gray)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
       flags=cv2.cv.CV_HAAR_SCALE_IMAGE
    )

    # Draw a rectangle around the faces
    if len(faces)>0:    
        for (x, y, w, h) in faces:
            fac = np.array(gray)[y:(y+h),x:(x+h)]
            X,X_normal = Pre_Process(fac)
            Probability = Net.predict_proba(X.astype(np.float32))
            prob = np.amax(Probability)
            index = np.argmax(Probability)
            #print Class
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2)
            cv2.putText(frame,str(map[index])+' '+str(round(prob*100,2) )+'%',(x,y+h), cv2.FONT_HERSHEY_DUPLEX,1,(255,255,255), 1,2)            
            #scipy.misc.toimage(fac).save(str(uuid.uuid4()) +'.jpg')

    # Display the resulting frame
        cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        


# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()
